In [1]:
# url_list = [('http://snap.stanford.edu/data/amazon/productGraph/categoryFiles/reviews_Musical_Instruments_5.json.gz', 'Musical_Instruments'),
#            ('http://snap.stanford.edu/data/amazon/productGraph/categoryFiles/reviews_Amazon_Instant_Video_5.json.gz', 'Instant_Videos')]
review_list = [('http://snap.stanford.edu/data/amazon/productGraph/categoryFiles/reviews_Books_5.json.gz','Books'),
       ('http://snap.stanford.edu/data/amazon/productGraph/categoryFiles/reviews_Electronics_5.json.gz','Electronics'),
       ('http://snap.stanford.edu/data/amazon/productGraph/categoryFiles/reviews_Movies_and_TV_5.json.gz','Movies_and_TV'),
       ('http://snap.stanford.edu/data/amazon/productGraph/categoryFiles/reviews_CDs_and_Vinyl_5.json.gz','CDs_and_Vinyl')]
#metadata_list = [('http://snap.stanford.edu/data/amazon/productGraph/categoryFiles/meta_Amazon_Instant_Video.json.gz', 'Instant_Video')]
metadata_list = [('http://snap.stanford.edu/data/amazon/productGraph/categoryFiles/meta_Electronics.json.gz', 'Electronics')]

stop_words_url = 'http://tacit.usc.edu/resources/stopwords_eng.txt'

In [2]:
import urllib
load_data = False
load_metadata = False
load_stop_words = False

if load_data:
  for url_tuple in review_list:
    url = url_tuple[0]
    folder_path = '/tmp/' + url_tuple[1] + '.json.gz'
    urllib.urlretrieve(url, folder_path)
    dbutils.fs.mv('file:' + folder_path, 'dbfs:/tmp/Data/' + url_tuple[1] + '.json.gz')
    display(dbutils.fs.ls("dbfs:/tmp/Data"))
    
if load_metadata:
  for url_tuple in metadata_list:
    url = url_tuple[0]
    folder_path = '/tmp/' + url_tuple[1] + '.json.gz'
    urllib.urlretrieve(url, folder_path)
    dbutils.fs.mv('file:' + folder_path, 'dbfs:/tmp/Metadata/' + url_tuple[1] + '.json.gz')
    display(dbutils.fs.ls("dbfs:/tmp/Metadata"))
    
if load_stop_words:
  urllib.urlretrieve(stop_words_url, '/tmp/stopwords_eng.txt')
  dbutils.fs.mv('file:/tmp/stopwords_eng.txt', 'dbfs:/tmp/stopwords_eng.txt')
  #   dbutils.fs.mv("file:/tmp/test.json.gz", "dbfs:/tmp/Data/test.json.gz")
  display(dbutils.fs.ls("dbfs:/tmp"))

In [3]:
from pyspark.sql.types import *

ratings_df_schema = StructType(
  [StructField('asin', StringType()),
   StructField('helpful', ArrayType(IntegerType())),
   StructField('overall', FloatType()),
   StructField('reviewText', StringType()),
   StructField('reviewTime', DateType()),
   StructField('reviewerID', StringType()),
   StructField('summary', StringType())]
)
# StructField('unixReviewTime', LongType())
# ['asin', 'description', 'title', 'categories']
metadata_df_schema = StructType(
  [StructField('asin', StringType()),
   StructField('description', StringType()),
   StructField('title', StringType()),
   StructField('categories', ArrayType(StringType()))]
)


In [4]:
#import 5-core review data

import_review_file = False

if import_review_file:
  import gzip
  from pyspark.sql import Row
  from collections import OrderedDict
  from datetime import datetime

  def parse_review(path):
    g = gzip.open(path, 'rb')
    for l in g:
      yield eval(l)

  def create_review_df(path):
    ratings = []
    for review in parse_review(path):
      ratings.append(review)

    #check if all entries contains a value for each key
  #   missing_keys = []
  #   all_keys = ['reviewerID', 'asin', 'reviewerName', 'helpful', 'unixReviewTime', 'reviewText', 'overall', 'reviewTime', 'summary']
  #   for i in range(len(ratings)):
  #     for j in range(len(all_keys)):
  #       if (all_keys[j] not in ratings[i].keys()):
  #         missing_keys.append(all_keys[j])
  #   print set(missing_keys)
    #not all entries have 'reviewerName'

    #remove revivewerName, unixreviewTime from all entries and convert reivewTime to datetime.date type
    for i in range(len(ratings)):
      ratings[i].pop('reviewerName', None)
      ratings[i].pop('unixReviewTime', None)
      ratings[i]['reviewTime'] = datetime.strptime(ratings[i]['reviewTime'], '%m %d, %Y')

    return sqlContext.createDataFrame(ratings, schema=ratings_df_schema)

  # books_df = create_review_df('/dbfs/tmp/Data/Books.json.gz')
  electronics_df = create_review_df('/dbfs/tmp/Data/Electronics.json.gz')
  # movies_df = create_review_df('/dbfs/tmp/Data/Movies_and_TV.json.gz')
  # cds_df = create_review_df('/dbfs/tmp/Data/CDs_and_Vinyl.json.gz')
  # instruments_df = create_review_df('/dbfs/tmp/Data/Musical_Instruments.json.gz')
  # instant_video_df = create_review_df('/dbfs/tmp/Data/Instant_Videos.json.gz')


  # print books_df.cache().count()
  print electronics_df.cache().count()
  # print movies_df.cache().count()
  # print cds_df.cache().count()
  # print instruments_df.cache().count()
  # print instant_video_df.cache().count()

  electronics_df.show(3, truncate=True)

In [5]:
a = 'one word'
b = 'a dskf sdfds sdf dsf one word'
print b.replace(a, "")


In [6]:

#import metadata files

import gzip
from pyspark.sql import Row
from collections import OrderedDict
from datetime import datetime

def parse_metadata(path):
  g = gzip.open(path, 'rb')
  for l in g:
    yield eval(l)
    
def create_metadata_df(path):
  metadata = []
  for entry in parse_metadata(path):
    metadata.append(entry)

  #find all keys
  all_keys = []
  for i in range(len(metadata)):
    for key in metadata[i].keys():
      if key not in all_keys:
        all_keys.append(key)
  print all_keys
  #['asin', 'categories', 'price', 'related'] in instant videos
  #['asin', 'imUrl', 'description', 'categories', 'title', 'related', 'price', 'salesRank', 'brand'] in electronics
  
  #check number of entries that contains each key
  key_count_dict = {}
  for key in all_keys:
    key_count_dict[key] = 0
  for i in range(len(metadata)):
    for key in all_keys:
      if key in metadata[i].keys():
        key_count_dict[key] += 1
  print key_count_dict  
#  {'asin': 498196, 'description': 459470, 'title': 491194, 'price': 389693, 'imUrl': 498021, 'related': 366959, 'salesRank': 128706, 'brand': 142532, 'categories': 498196} for electronics
  discard_missing_info_col = True
  cols_to_keep = ['asin', 'description', 'title', 'categories']
  filtered_metadata = []
  for i in range(len(metadata)):
    keep_col = True
    if discard_missing_info_col:
      #only retain items that contain info for all columns
      cols = [x for x in cols_to_keep if x in metadata[i].keys()]
      if len(cols) != len(cols_to_keep):
        keep_col = False
    if keep_col:
      temp_dict = {}
      for col in cols_to_keep: 
        if col not in metadata[i].keys():
          temp_dict[col] = None
        elif col == 'categories':#given as array of array so only keeep the inner array
          temp_dict[col] = metadata[i]['categories'][0]
        else:
          temp_dict[col] = metadata[i][col]
      filtered_metadata.append(temp_dict)

  return sqlContext.createDataFrame(filtered_metadata, schema = metadata_df_schema)#schema=metadata_df_schema
  

# books_df = create_review_df('/dbfs/tmp/Metadata/Books.json.gz')
electronics_meta_df = create_metadata_df('/dbfs/tmp/Metadata/Electronics.json.gz')
electronics_meta_df.cache().count()
#instant_video_meta_df = create_metadata_df('/dbfs/tmp/Metadata/Instant_Video.json.gz')
# movies_df = create_review_df('/dbfs/tmp/Metadata/Movies_and_TV.json.gz')
# cds_df = create_review_df('/dbfs/tmp/Metadata/CDs_and_Vinyl.json.gz')
# instruments_df = create_review_df('/dbfs/tmp/Metadata/Musical_Instruments.json.gz')
# instant_video_df = create_review_df('/dbfs/tmp/Metadata/Instant_Videos.json.gz')


electronics_meta_df.show(20)



In [7]:
def find_brands(path):
  brands = []
  count = 0
  for entry in parse_metadata(path):
    if 'brand' in entry.keys() and entry['brand'] not in brands:
      brands.append(entry['brand'])
  return brands
   
brands_stop_words = find_brands('/dbfs/tmp/Metadata/Electronics.json.gz')
print len(brands_stop_words)


In [8]:
print brands_stop_words[:12]

In [9]:
print electronics_meta_df.count()
#489187 when including all entries
#453246 when discarding entries with null elements in either category, descripton and title (discard_missing_info_col = True)

In [10]:
#print # of distinct categories among all products
print electronics_meta_df.select('categories').distinct().count()

In [11]:
#import stopwords
stopwords_path = 'dbfs:/tmp/stopwords_eng.txt'
stopwords_rdd = sc.textFile(stopwords_path)
stopwords = stopwords_rdd.collect()
print len(stopwords)
print stopwords


In [12]:
#parse each column by itself instead
# feature hashing
#convert each row of metadata_df to list of tuple format 
from pyspark.sql.functions import udf, array, split
import re

def parse_row(col):
  if col == None:
    return [(0, 'EmptyString')]
  elif isinstance(col, basestring):
    #remove brands first
    for brand in brands_stop_words:
      if brand in col:
        col = col.replace(brand, "")
        break
    words = re.split(r'\W+',col)
  elif type(col) is list:
    words = col
   
  #assign featureID to its index in all_words (e.g. des_words_ID = 0, title_words_ID = 1, etc.)
  tuple_list = []
  for word in words:
    lower_case_word = word.lower()
    if (lower_case_word not in stopwords):
      tuple_list.append((0, lower_case_word))
  return tuple_list


if not electronics_meta_df.is_cached:
  electronics_meta_df.cache().count()

  
parse_row_udf = udf(parse_row, ArrayType(StructType([StructField('_1', LongType()),
                                                         StructField('_2', StringType())])))
tuple_electronics_meta_df = electronics_meta_df.select(electronics_meta_df.asin, \
                                                       parse_row_udf(electronics_meta_df.description).alias('description_features'), \
                                                       parse_row_udf(electronics_meta_df.title).alias('title_features'), \
                                                       parse_row_udf(electronics_meta_df.categories).alias('category_features'))
                                                                                                                    
if electronics_meta_df.is_cached:
  electronics_meta_df.unpersist()
if not tuple_electronics_meta_df.is_cached:
  tuple_electronics_meta_df.cache().show(3)


In [13]:
#check for null elements for each feature
print tuple_electronics_meta_df.where(tuple_electronics_meta_df.description_features.isNull()).count()
print tuple_electronics_meta_df.where(tuple_electronics_meta_df.title_features.isNull()).count()
print tuple_electronics_meta_df.where(tuple_electronics_meta_df.category_features.isNull()).count()


In [14]:
from collections import defaultdict
import hashlib

def hash_function(raw_feats, num_buckets, print_mapping=False):
    """Calculate a feature dictionary for an observation's features based on hashing.

    Note:
        Use print_mapping=True for debug purposes and to better understand how the hashing works.

    Args:
        raw_feats (list of (int, str)): A list of features for an observation.  Represented as
            (featureID, value) tuples.
        num_buckets (int): Number of buckets to use as features.
        print_mapping (bool, optional): If true, the mappings of featureString to index will be
            printed.

    Returns:
        dict of int to float:  The keys will be integers which represent the buckets that the
            features have been hashed to.  The value for a given key will contain the count of the
            (featureID, value) tuples that have hashed to that key.
    """
    mapping = { category + ':' + str(ind):
                int(int(hashlib.md5(category + ':' + str(ind)).hexdigest(), 16) % num_buckets)
                for ind, category in raw_feats}
    if(print_mapping): print mapping

    def map_update(l, r):
        l[r] += 1.0
        return l

    sparse_features = reduce(map_update, mapping.values(), defaultdict(float))
    return dict(sparse_features)

In [15]:
from pyspark.ml.linalg import Vectors, VectorUDT
import numpy as np


num_hash_buckets_description = 2 ** 8
num_hash_buckets_title = 2 ** 9
num_hash_buckets_category = 2 ** 9

# UDF that returns a vector of hashed features given an Array of tuples

def tuples_to_hash_features_desc(col):
  return Vectors.sparse(num_hash_buckets_description, hash_function(col, num_hash_buckets_description))
tuples_to_hash_features_desc_udf = udf(tuples_to_hash_features_desc, VectorUDT())

def tuples_to_hash_features_title(col):
  return Vectors.sparse(num_hash_buckets_title, hash_function(col, num_hash_buckets_title))
tuples_to_hash_features_title_udf = udf(tuples_to_hash_features_title, VectorUDT())

def tuples_to_hash_features_category(col):
  return Vectors.sparse(num_hash_buckets_category, hash_function(col, num_hash_buckets_category))
tuples_to_hash_features_category_udf = udf(tuples_to_hash_features_category, VectorUDT())


def add_hashed_features(df):
    """Return a DataFrame with labels and hashed features.

    Note:
        Make sure to cache the DataFrame that you are returning.

    Args:
        df (DataFrame with 'tuples' column): A DataFrame containing the tuples to be hashed.

    Returns:
        DataFrame: A DataFrame with a 'label' column and a 'features' column that contains a
            SparseVector of hashed features.
    """
    return (df.select(df.asin, \
                      tuples_to_hash_features_desc_udf(df.description_features).alias('description_hash_features'), \
                      tuples_to_hash_features_title_udf(df.title_features).alias('title_hash_features'), \
                      tuples_to_hash_features_category_udf(df.category_features).alias('category_hash_features')))

hash_electronics_meta_df = add_hashed_features(tuple_electronics_meta_df)

tuple_electronics_meta_df.unpersist()
hash_electronics_meta_df.cache().show(3)

In [16]:
print hash_electronics_meta_df.where(hash_electronics_meta_df.description_hash_features.isNull()).count()
print hash_electronics_meta_df.where(hash_electronics_meta_df.title_hash_features.isNull()).count()
print hash_electronics_meta_df.where(hash_electronics_meta_df.category_hash_features.isNull()).count()

In [17]:
from pyspark.ml.linalg import Vectors, VectorUDT, SparseVector


# combine_hash_features_udf = udf(lambda x1:x1, VectorUDT())
def combine_hash_features(col1, col2, col3):
#   if isinstance(col1,SparseVector) and isinstance(col2,SparseVector) and isinstance(col3,SparseVector):
  combined_array = np.concatenate((col1.toArray(),col2.toArray(), col3.toArray()))
  sparse_vec = {i:combined_array[i] for i in np.nonzero(combined_array)[0]}
  return Vectors.sparse(len(combined_array), sparse_vec)

combine_hash_features_udf = udf(combine_hash_features, VectorUDT())

hash_combined_electronics_meta_df = hash_electronics_meta_df.select(hash_electronics_meta_df.asin, \
                                                                    combine_hash_features_udf(hash_electronics_meta_df.description_hash_features, \
                                                                                              hash_electronics_meta_df.title_hash_features, \
                                                                                              hash_electronics_meta_df.category_hash_features).alias('features'))
hash_electronics_meta_df.unpersist()
hash_combined_electronics_meta_df.cache().show(3)


In [18]:
from pyspark.ml.clustering import KMeans
from pyspark.ml.linalg import Vectors

num_clusters_list = [50, 100, 250, 500, 1000, 2000]
scores = []
models = []
for num_clusters in num_clusters_list:
  kmeans = KMeans(k=num_clusters, seed=1, featuresCol="features")
  model = kmeans.fit(hash_combined_electronics_meta_df)
  models.append(model)
  score = model.computeCost(hash_combined_electronics_meta_df)
  scores.append((num_clusters, score))

print scores
#256, 512, 512
# [(50, 30603181.219049413), (100, 29854189.077426285), (250, 28420735.469728597), (500, 27040367.94809037), (1000, 25572329.794782765), (2000, 23893421.253901474)]

In [19]:
print model.summary.k

In [20]:

import plotly.offline as pyo
import plotly.graph_objects as go


In [21]:
import plotly.offline as pyo
import plotly.graph_objs as go
import matplotlib.pyplot as plt


x_val = [x[0] for x in scores]
y_val = [x[1] for x in scores]

trace = go.Scatter(
  x = x_val,
  y = y_val
)

figure = go.Figure(data=go.Data([trace]), layout=go.Layout())
displayHTML(pyo.plot(figure, output_type='div'))

In [22]:
for model in models:
  filename = 'Kmeans_' + str(num_hash_buckets_description) + '_' + str(num_hash_buckets_title) + '_' + str(num_hash_buckets_category) + '_' + str(model.summary.k)
  model.save("dbfs:/tmp/Clustering_results/" + filename)


In [23]:
dbutils.fs.ls("dbfs:/tmp/Clustering_results")

In [24]:
from pyspark.ml.clustering import KMeans, KMeansModel
best_model = KMeansModel.load("dbfs:/tmp/Clustering_results/Kmeans_256_512_512_100/")
transformed = best_model.transform(hash_combined_electronics_meta_df)
cluster_size = []
#for k in range(best_model.summary.k):
 # cluster_size.append((k, transformed.filter(transformed.prediction == k).count()))

# rows = transformed.collect()

In [25]:
transformed.show(5)

In [26]:
rows[:5]